In [1]:
import pandas as pd
import spacy
import numpy as np
from textblob import TextBlob
import re
import string
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, ENGLISH_STOP_WORDS
from pymongo import MongoClient
from pprint import pprint

2021-12-16 13:51:29.566311: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-16 13:51:29.566370: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
client = MongoClient("mongodb://localhost:27017")
client.list_database_names()

['admin', 'books', 'coffee_reviews', 'config', 'events', 'local', 'outings']

In [3]:
reviews_mongo = client.coffee_reviews
reviews_mongo.list_collection_names()

['reviews_col']

In [13]:
def reviews_to_mongo(file):
    df = pd.read_csv(file)
    df.drop(columns='Unnamed: 0', inplace=True)
    
    rev_dict = df.to_dict(orient='list')
    
    reviews_mongo.reviews_col.insert_many([rev_dict], ordered=True)
    
    return 'download complete'

In [20]:
files = ['Anchorhead Coffee_reviews.csv',
        'Coffeeholic House_reviews.csv',
        'Espresso Vivace Roasteria_reviews.csv',
        'Milstead_reviews.csv',
        'Moonshot Coffee_reviews.csv',
        'Moore Coffee Shop_reviews.csv',
        'Santo Coffee_reviews.csv',
        'Storyville Coffee Company_reviews.csv',
        'Sugar Bakery & Coffeehouse_reviews.csv']

In [17]:
for file in files:
    reviews_to_mongo(file)

In [31]:
df2 = pd.DataFrame(list(reviews_mongo.reviews_col.aggregate([{'$unwind': '$reviews'}])))
df2.columns = ['ID', 'shop_name', 'reviews']

In [32]:
files2 = ['Lighthouse Roasters',
          'Anchorhead Coffee',
        'Coffeeholic House',
        'Espresso Vivace Roasteria',
        'Milstead',
        'Moonshot Coffee',
        'Moore Coffee Shop',
        'Santo Coffee',
        'Storyville Coffee Company',
        'Sugar Bakery & Coffeehouse']

In [33]:
for i in range(0,10):
    lh = list(df2.ID.unique())[i]
    name = files2[i]
    df2.replace({lh: f'{name}'}, inplace=True)
df2.ID.unique()

array(['Lighthouse Roasters', 'Anchorhead Coffee', 'Coffeeholic House',
       'Espresso Vivace Roasteria', 'Milstead', 'Moonshot Coffee',
       'Moore Coffee Shop', 'Santo Coffee', 'Storyville Coffee Company',
       'Sugar Bakery & Coffeehouse'], dtype=object)

In [35]:
df2.drop(columns='shop_name', inplace=True)

In [36]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6057 entries, 0 to 6056
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   ID       6057 non-null   object
 1   reviews  6057 non-null   object
dtypes: object(2)
memory usage: 94.8+ KB


In [37]:
df2.to_csv('Seattle_Coffee_Reviews.csv')